# Get predictions + performance of best Ridge model on heldout test set

In [2]:
# import numpy as np
import pandas as pd
import yaml
import importlib

# scikit-learn
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# own module
from brain_age_prediction import utils, sklearn_utils
importlib.reload(utils)
importlib.reload(sklearn_utils)

<module 'brain_age_prediction.sklearn_utils' from '/home/laurar/brain_age_1D-CNN/src/brain_age_prediction/sklearn_utils.py'>

In [3]:
# save path for predictions
preds_save_path = '/ritter/share/projects/laura_riedel_thesis/predictions/'

Retrain model with best performing alpha:

In [4]:
seed = 43
utils.make_reproducible(random_state=seed)
# get data
X_train, y_train, _, _, X_test, y_test = sklearn_utils.access_dataset(schaefer_variant='7n100p',no_0=True,normalise=True)
X_heldout, y_heldout = sklearn_utils.access_datasplit(split='heldout_test',schaefer_variant='7n100p',no_0=True,normalise=True)

# load ridge config
with open('parameters/7n100p_alpha-2636.yaml', 'r') as f:
    config = yaml.safe_load(f)

# define model
ridge_model = Ridge(alpha=config['parameters']['alpha'],
                    random_state=seed)
        
# train
ridge_model.fit(X_train, y_train)

Ridge(alpha=2636.6508987303555, random_state=43)

Confirm model performs the same as in alpha test:

In [5]:
y_pred_test = ridge_model.predict(X_test)
print('test_mae',mean_absolute_error(y_test, y_pred_test))
print('test_loss',mean_squared_error(y_test, y_pred_test))
print('test_r2',r2_score(y_test, y_pred_test))

test_mae 4.784899104197659
test_loss 34.68214615333583
test_r2 0.4414223029564829


Determine model performance on heldout test set:

In [7]:
y_pred_heldout = ridge_model.predict(X_heldout)
print('test_mae',mean_absolute_error(y_heldout, y_pred_heldout))
print('test_loss',mean_squared_error(y_heldout, y_pred_heldout))
print('test_r2',r2_score(y_heldout, y_pred_heldout))

test_mae 4.684324441528321
test_loss 33.351581432477644
test_r2 0.43730077958761704


In [8]:
heldout_df = pd.read_csv('../../data/schaefer/overview_heldout_test_set_100-500p.csv')
heldout_df['predicted_age_ridge'] = y_pred_heldout
heldout_df

,eid,age,predicted_age_ridge
0,1000030,65,65.442215
1,1000059,68,65.902206
2,1000077,58,60.723911
3,1000100,57,51.595612
4,1000202,67,62.639080
...,...,...,...
4995,2035762,73,70.628128
4996,2264650,61,60.719040
4997,2329354,68,66.051826
4998,2457356,60,58.524540


In [9]:
# save prediction
heldout_df.to_csv(preds_save_path+'ridge_preds_100-500p.csv', index=False)